In [115]:
# common recipe to get access to the source inside src
import os
import sys
module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)
##########################################################

import numpy as np
from pathlib import Path

from models.CaptionModalityClassifier import CaptionModalityClassifier # caption modality classifier
from dataset.MultimodalityDataModule import MultimodalityDataModule    # pytorch lightning data module for captions and images
from utils.caption_utils import load_glove_matrix                      # utils for embeddings

from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import EarlyStopping
from pytorch_lightning.loggers import WandbLogger

In [116]:
MAX_NUMBER_WORDS = 20000       # number of words to consider from embeddings vocabulary
MAX_WORDS_PER_SENTENCE = 300   # sentence maximum length
WORD_DIMENSION = 300           # number of features per embedding
NUM_CLASSES = 4                # 4 microscopy classes
NUM_FILTERS = 100              # number of filters use in the ConvText module

BASE_PATH = Path('/workspace/data')
DATA_PATH = BASE_PATH / 'multimodality_classification.csv'
EMBEDDINGS = BASE_PATH / 'embeddings'
OUTPUT_DIR = Path('./outputs')
BASE_IMG_DIR = BASE_PATH       # the image path in the CSV file are relative to this directory
BATCH_SIZE = 32
LR = 1e-4
PROJECT = 'biomedical-multimodal'

NUM_WORKERS = 72               # workers on the cluster, normally... 4?

In [117]:
import torch
from transformers import BertTokenizer, BertModel
from sklearn.preprocessing import LabelEncoder

#model_name = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract"
model_name = 'bert-base-uncased'

tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=True)

In [118]:
import pandas as pd
df = pd.read_csv(DATA_PATH, sep='\t')
df_train = df[df['SET'] == 'TRAIN']
df_val = df[df['SET'] == 'VAL']
df_test = df[df['SET'] == 'TEST']

In [119]:
len(df_train) + len(df_val)

2330

In [39]:
subset = df[['CAPTION', 'MODALITY']]
new_df = subset[['CAPTION', 'MODALITY']].drop_duplicates(keep='first', inplace=False)
new_df

,CAPTION,MODALITY
0,Scanning electron microscope images of the TiO...,DMEL
5,Trophoblast differentiation from embryoid bodi...,DMEL
6,Transfection of SUNE1 cell with LMP1 induces...,DMEL
7,Morphology and CD90 staining . M Î¦ visualiza...,DMEL
9,Regulation of TGF- Î² 1-induced changes in cel...,DMEL
...,...,...
3191,Author/co-authorship graphs for nine pilot cen...,DMLI
3192,Improving the FP rate of the detection results...,DMFL
3196,"Parameters used: (b) n = 60, Î± = 130, Î±...",DMTR
3198,FISH technique: A single-stranded probe was us...,DMFL


In [36]:
duplicateDFRow[duplicateDFRow.duplicated(['CAPTION'])]

,CAPTION,MODALITY
227,Trophoblast differentiation from embryoid bodi...,DMFL
229,Transfection of SUNE1 cell with LMP1 induces...,DMFL
243,BALF fibroblasts in patients with mild asthma ...,DMFL
314,CLIC1 in T84 cells . Cells were grown to confl...,DMFL
317,Abnormalities observed during meiosis of Mer 3...,DMFL
...,...,...
3100,Tyrosine hydroxylase and NeuN cell counting in...,DMLI
3104,Quantum dot labeled human mesenchymal stem cel...,DMFL
3105,Quantum dot labeled human mesenchymal stem cel...,DMLI
3127,Immunolocalization of Cs CB in adult worm of ...,DMFL


In [28]:
subset.head(6)

,CAPTION,MODALITY
0,Scanning electron microscope images of the TiO...,DMEL
1,Scanning electron microscope images of the TiO...,DMEL
2,Scanning electron microscope images of the TiO...,DMEL
3,Scanning electron microscope images of the TiO...,DMEL
4,Scanning electron microscope images of the TiO...,DMEL
5,Trophoblast differentiation from embryoid bodi...,DMEL


In [124]:
def get_input_ids(captions, labels):
    input_ids = []
    attention_masks = []

    for caption in captions:
        # `encode_plus` will:
        #   (1) Tokenize the sentence.
        #   (2) Prepend the `[CLS]` token to the start.
        #   (3) Append the `[SEP]` token to the end.
        #   (4) Map tokens to their IDs.
        #   (5) Pad or truncate the sentence to `max_length`
        #   (6) Create attention masks for [PAD] tokens.
        encoded_dict = tokenizer.encode_plus(
                            caption,                   # Sentence to encode.
                            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                            max_length = 500,          # Pad & truncate all sentences.
                            pad_to_max_length = True,
                            return_attention_mask = True,   # Construct attn. masks.
                            return_tensors = 'pt',     # Return pytorch tensors.
                            truncation=True
                       )

        # Add the encoded sentence to the list.    
        input_ids.append(encoded_dict['input_ids'])

        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(encoded_dict['attention_mask'])

    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels)

    # Print sentence 0, now as a list of IDs.
    print('Original: ', captions[0], labels[0])
    print('Token IDs:', input_ids[0])
    
    return input_ids, attention_masks, labels

In [64]:
new_captions = new_df['CAPTION'].values
new_labels = new_df['MODALITY'].values
new_labels = le.transform(new_labels)
new_input_ids, new_attention_masks, new_labels = get_input_ids(new_captions, new_labels)

Original:  Scanning electron microscope images of the TiO 2  nanoparticles . (A) C150, (B) C200, (C) EDS elemental spectrum of C200, (D)  S. aureus  and UV100, (E)  S. aureus  and C150, and (F)  S. aureus  and C200. Scale bars: 100 nm. tensor(0)
Token IDs: tensor([  101, 13722, 10496, 24635,  4871,  1997,  1996, 14841,  2080,  1016,
        28991, 19362,  4588,  4244,  1012,  1006,  1037,  1007, 27723, 12376,
         1010,  1006,  1038,  1007, 29248,  8889,  1010,  1006,  1039,  1007,
        11985, 19529,  8674,  1997, 29248,  8889,  1010,  1006,  1040,  1007,
         1055,  1012,  8740, 23446,  1998, 23068, 18613,  1010,  1006,  1041,
         1007,  1055,  1012,  8740, 23446,  1998, 27723, 12376,  1010,  1998,
         1006,  1042,  1007,  1055,  1012,  8740, 23446,  1998, 29248,  8889,
         1012,  4094,  6963,  1024,  2531, 13221,  1012,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0

In [125]:
train_captions = df_train['CAPTION'].values
train_labels   = df_train['MODALITY'].values

le = LabelEncoder()
le.fit(train_labels)
train_labels = le.transform(train_labels)

val_captions = df_val['CAPTION'].values
val_labels   = df_val['MODALITY'].values
val_labels   = le.transform(val_labels)

test_captions = df_test['CAPTION'].values
test_labels   = df_test['MODALITY'].values
test_labels   = le.transform(test_labels)

train_input_ids, train_attention_masks, train_labels = get_input_ids(train_captions, train_labels)
val_input_ids, val_attention_masks, val_labels = get_input_ids(val_captions, val_labels)
test_input_ids, test_attention_masks, test_labels = get_input_ids(test_captions, test_labels)

Original:  Scanning electron microscope images of the TiO 2  nanoparticles . (A) C150, (B) C200, (C) EDS elemental spectrum of C200, (D)  S. aureus  and UV100, (E)  S. aureus  and C150, and (F)  S. aureus  and C200. Scale bars: 100 nm. tensor(0)
Token IDs: tensor([  101, 13722, 10496, 24635,  4871,  1997,  1996, 14841,  2080,  1016,
        28991, 19362,  4588,  4244,  1012,  1006,  1037,  1007, 27723, 12376,
         1010,  1006,  1038,  1007, 29248,  8889,  1010,  1006,  1039,  1007,
        11985, 19529,  8674,  1997, 29248,  8889,  1010,  1006,  1040,  1007,
         1055,  1012,  8740, 23446,  1998, 23068, 18613,  1010,  1006,  1041,
         1007,  1055,  1012,  8740, 23446,  1998, 27723, 12376,  1010,  1998,
         1006,  1042,  1007,  1055,  1012,  8740, 23446,  1998, 29248,  8889,
         1012,  4094,  6963,  1024,  2531, 13221,  1012,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0

Original:  Confocal image of 5-HT 2A  receptors in the hippocampus of control and Experimental rats using immunofluorescent 5-HT 2A  receptor specific primary antibody and FITC as secondary antibody . The pixel intensity of The pixel intensity of control - 384132  Â±  1454, diabetic - 133475  Â±  1431  a , diabetic+Insulin - 229123  Â±  1453  a, c , diabetic+pyridoxine - 151012  Â±  2662  a, b , diabetic+insulin+pyridoxine - 398791  Â±  2105  c , diabetic+ Aegle marmelose -  222921  Â±  1097 a, c  and diabetic+ Aegle marmelose+ pyridoxine - 314997  Â±  1084  c . Values are mean  Â±  S.E.M of 4-6 rats in each group. Each group consists of 6-8 rats.  a  p  <  0.001 when compared to control group;  b  p  <  0.05,  c  p  <  0.001 when compared to diabetic group. There was significant reversal of 5-HT 2A  receptor to near control level on treatment with pyridoxine alone and in combination therapy with insulin and  Aegle marmelose  compared to diabetic rats. Arrow in white shows 5-HT 2A  rec

In [126]:
from torch.utils.data import TensorDataset
train_dataset = TensorDataset(train_input_ids, train_attention_masks, train_labels)
val_dataset   = TensorDataset(val_input_ids, val_attention_masks, val_labels)
test_dataset   = TensorDataset(test_input_ids, test_attention_masks, test_labels)

print('{:>5,} training samples'.format(len(train_dataset)))
print('{:>5,} validation samples'.format(len(val_dataset)))
print('{:>5,} test samples'.format(len(test_dataset)))

1,864 training samples
  466 validation samples
  873 test samples


In [83]:
# for new
# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(new_input_ids, new_attention_masks, new_labels)
batch_size=32

from sklearn.model_selection import train_test_split

train_idx, valid_idx= train_test_split(np.arange(len(new_labels)),
                        test_size=0.1,
                        shuffle=True,
                        stratify=new_labels,
                        random_state=43)

train_sampler = torch.utils.data.SubsetRandomSampler(train_idx)
valid_sampler = torch.utils.data.SubsetRandomSampler(valid_idx)

train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
validation_dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler)

print('{:>5,} training samples'.format(len(train_idx)))
print('{:>5,} validation samples'.format(len(valid_idx)))

  890 training samples
   99 validation samples


In [97]:
new_df.head()

,level_0,index,CAPTION,MODALITY,SET,rowIndex
0,0,0,Scanning electron microscope images of the TiO...,DMEL,TRAIN,0
1,1,5,Trophoblast differentiation from embryoid bodi...,DMEL,TRAIN,1
2,2,6,Transfection of SUNE1 cell with LMP1 induces...,DMEL,TRAIN,2
3,3,7,Morphology and CD90 staining . M Î¦ visualiza...,DMEL,TRAIN,3
4,4,9,Regulation of TGF- Î² 1-induced changes in cel...,DMEL,TRAIN,4


In [99]:
new_df['SET'] = 'TRAIN'
#new_df = new_df.reset_index()
#def rowIndex(row): return row.name
#new_df['rowIndex'] = new_df.apply(rowIndex, axis=1)
new_df['SET'] = new_df.apply(lambda x: 'VALID' if x['rowIndex'] in valid_idx else 'TRAIN', axis=1)
new_df.head()

,level_0,index,CAPTION,MODALITY,SET,rowIndex
0,0,0,Scanning electron microscope images of the TiO...,DMEL,TRAIN,0
1,1,5,Trophoblast differentiation from embryoid bodi...,DMEL,TRAIN,1
2,2,6,Transfection of SUNE1 cell with LMP1 induces...,DMEL,TRAIN,2
3,3,7,Morphology and CD90 staining . M Î¦ visualiza...,DMEL,TRAIN,3
4,4,9,Regulation of TGF- Î² 1-induced changes in cel...,DMEL,TRAIN,4


In [101]:
new_df[new_df['SET']=='TRAIN'].shape

(890, 6)

In [107]:
to_save_df = new_df[['CAPTION', 'MODALITY', 'SET']]
to_save_df['PATH'] = 'not used'
to_save_df.head()



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,CAPTION,MODALITY,SET,PATH
0,Scanning electron microscope images of the TiO...,DMEL,TRAIN,not used
1,Trophoblast differentiation from embryoid bodi...,DMEL,TRAIN,not used
2,Transfection of SUNE1 cell with LMP1 induces...,DMEL,TRAIN,not used
3,Morphology and CD90 staining . M Î¦ visualiza...,DMEL,TRAIN,not used
4,Regulation of TGF- Î² 1-induced changes in cel...,DMEL,TRAIN,not used


In [110]:
test_df = df[df['SET'] == 'TEST']
test_df = test_df[['CAPTION', 'MODALITY', 'SET', 'PATH']]
test_df.head()

,CAPTION,MODALITY,SET,PATH
2330,Confocal image of 5-HT 2A receptors in the hi...,DMFL,TEST,subfigure-classification/2016/test/1423-0127-1...
2331,Confocal image of 5-HT 2A receptors in the hi...,DMFL,TEST,subfigure-classification/2016/test/1423-0127-1...
2332,Confocal image of 5-HT 2A receptors in the hi...,DMFL,TEST,subfigure-classification/2016/test/1423-0127-1...
2333,Confocal image of 5-HT 2A receptors in the hi...,DMFL,TEST,subfigure-classification/2016/test/1423-0127-1...
2334,Confocal image of 5-HT 2A receptors in the hi...,DMFL,TEST,subfigure-classification/2016/test/1423-0127-1...


In [113]:
concat_df = pd.concat([to_save_df, test_df])
print(concat_df.shape)
concat_df.head()

(1862, 4)


,CAPTION,MODALITY,SET,PATH
0,Scanning electron microscope images of the TiO...,DMEL,TRAIN,not used
1,Trophoblast differentiation from embryoid bodi...,DMEL,TRAIN,not used
2,Transfection of SUNE1 cell with LMP1 induces...,DMEL,TRAIN,not used
3,Morphology and CD90 staining . M Î¦ visualiza...,DMEL,TRAIN,not used
4,Regulation of TGF- Î² 1-induced changes in cel...,DMEL,TRAIN,not used


In [114]:
concat_df.to_csv('./multimodal_simplified.csv', sep='\t')

In [127]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 32

train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

test_dataloader = DataLoader(
            test_dataset, # The validation samples.
            sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [128]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

In [129]:
model = BertForSequenceClassification.from_pretrained(
    model_name, # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 4,                     
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [130]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 201 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (30522, 768)
bert.embeddings.position_embeddings.weight                (512, 768)
bert.embeddings.token_type_embeddings.weight                (2, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [131]:
optimizer = AdamW(model.parameters(),
                  lr = 5e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [132]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 4

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [133]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [134]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [135]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

device = torch.device("cuda:0")

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        loss, logits = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            (loss, logits) = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.4f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 4 ========
Training...
  Batch    40  of     59.    Elapsed: 0:00:38.

  Average training loss: 1.07
  Training epoch took: 0:00:56

Running Validation...
  Accuracy: 0.7190
  Validation Loss: 0.77
  Validation took: 0:00:05

======== Epoch 2 / 4 ========
Training...
  Batch    40  of     59.    Elapsed: 0:00:39.

  Average training loss: 0.58
  Training epoch took: 0:00:56

Running Validation...
  Accuracy: 0.8005
  Validation Loss: 0.61
  Validation took: 0:00:05

======== Epoch 3 / 4 ========
Training...
  Batch    40  of     59.    Elapsed: 0:00:39.

  Average training loss: 0.41
  Training epoch took: 0:00:56

Running Validation...
  Accuracy: 0.8287
  Validation Loss: 0.54
  Validation took: 0:00:05

======== Epoch 4 / 4 ========
Training...
  Batch    40  of     59.    Elapsed: 0:00:39.

  Average training loss: 0.33
  Training epoch took: 0:00:56

Running Validation...
  Accuracy: 0.8387
  Validation Loss: 0.53
  Validation took: 0:00:05

Training complete!


In [136]:
import pandas as pd

# Display floats with two decimal places.
pd.set_option('precision', 4)

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')

# A hack to force the column headers to wrap.
#df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

# Display the table.
df_stats

,Training Loss,Valid. Loss,Valid. Accur.,Training Time,Validation Time
epoch,,,,,
1,1.0716,0.7675,0.7190,0:00:56,0:00:05
2,0.5846,0.6066,0.8005,0:00:56,0:00:05
3,0.4088,0.5441,0.8287,0:00:56,0:00:05
4,0.3275,0.5348,0.8387,0:00:56,0:00:05


In [138]:
model.eval()

# Tracking variables 
total_eval_accuracy = 0
total_eval_loss = 0
nb_eval_steps = 0

# Evaluate data for one epoch
for batch in test_dataloader:

    # Unpack this training batch from our dataloader. 
    #
    # As we unpack the batch, we'll also copy each tensor to the GPU using 
    # the `to` method.
    #
    # `batch` contains three pytorch tensors:
    #   [0]: input ids 
    #   [1]: attention masks
    #   [2]: labels 
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    # Tell pytorch not to bother with constructing the compute graph during
    # the forward pass, since this is only needed for backprop (training).
    with torch.no_grad():        

        # Forward pass, calculate logit predictions.
        # token_type_ids is the same as the "segment ids", which 
        # differentiates sentence 1 and 2 in 2-sentence tasks.
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # Get the "logits" output by the model. The "logits" are the output
        # values prior to applying an activation function like the softmax.
        (loss, logits) = model(b_input_ids, 
                               token_type_ids=None, 
                               attention_mask=b_input_mask,
                               labels=b_labels)

    # Accumulate the validation loss.
#     total_eval_loss += loss.item()

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Calculate the accuracy for this batch of test sentences, and
    # accumulate it over all batches.
    total_eval_accuracy += flat_accuracy(logits, label_ids)


# Report the final accuracy for this validation run.
avg_val_accuracy = total_eval_accuracy / len(test_dataloader)
print("  Accuracy: {0:.4f}".format(avg_val_accuracy))


  Accuracy: 0.6566


"uncased bert 84.33"